# Constructing Viginia COVID Statistic DataFrames 
#### The goal is to connect to the official Virginia Health Department API for current COVID statistics, and to create two dataframes:
1. A dataframe with the total COVID cases, hospitalizations, and deaths by county/city, both overall and per 1000 people
2. A dataframe with the trend of these statistics, defined as the total number in the last two weeks relative to the total number during the two weeks prior to that

First we load the following packages:

In [1]:
import numpy as np
import pandas as pd
import requests
import json
from datetime import timedelta

#The following packages are for making visualizations and dashbords
import plotly.express as px
import dash
from dash import dcc
from dash import html
from dash import dash_table
from dash.dependencies import Input, Output

### Accessing the VDH COVID API
We will be using the Virginia Department of Health's API endpoint for the official reported numbers on the number of COVID hospitalizations and deaths for each county and independent city in Virginia. Here is the endpoint, which provides the data in JSON format: https://data.virginia.gov/resource/bre9-aqqr.json

The data are organized such that one record is one locality on one day. The fields within each record are

* `report_date`: Date in YYYY-MM-DDTHH:MM:SS.SSS format

* `fips`: A unique numeric ID for the locality represented in the record. 51 refers to Virginia, and the last 3 digits vary depending on the location

* `locality`: The name of the county or independent city for the record

* `vdh_health_district`: The name of the regional health district (the entity chiefly responsible for keeping statistics and administering COVID vaccines) that the locality is located in

* `total_cases`: the total official number of COVID cases on the record's date since the beginning of data collection on COVID cases (although be careful, "official" does not mean true -- these numbers are almost certainly significant undercounts because many cases went unreported)

* `hospitalizations`: the total number of hospitalizations due to COVID on the record's date since the beginning of data collection

* `deaths`: the total number of deaths due to COVID on the record's date since the beginning of data collection

The API that provides the data only provides 1000 records, by default. However we can change this by specifying the $limit parameter (see the documentation here: https://dev.socrata.com/docs/paging.html). Anything more than 200,000 should be more than sufficient for our purposes.

In [2]:
endpoint = 'https://data.virginia.gov/resource/bre9-aqqr.json'
mypars = {'$limit': 200000}
headers = {'User-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:99.0) Gecko/20100101 Firefox/99.0'}
r = requests.get(endpoint, params=mypars, headers=headers)
r

<Response [200]>

### A dataframe with the total COVID cases, hospitalizations, and deaths by county/city, both overall and per 1000 people
We parse the JSON output of the API call, and we change the data types of the columns:

In [3]:
cases = pd.json_normalize(json.loads(r.text))
cases['report_date'] = pd.to_datetime(cases['report_date'])
cases['total_cases'] = cases['total_cases'].astype('int')
cases['hospitalizations'] = cases['hospitalizations'].astype('int')
cases['deaths'] = cases['deaths'].astype('int')
cases

,report_date,fips,locality,vdh_health_district,total_cases,hospitalizations,deaths
0,2022-08-08,51001,Accomack,Eastern Shore,7866,424,111
1,2022-08-08,51003,Albemarle,Blue Ridge,21726,491,176
2,2022-08-08,51005,Alleghany,Alleghany,3781,99,101
3,2022-08-08,51007,Amelia,Piedmont,3014,171,59
4,2022-08-08,51009,Amherst,Central Virginia,8225,356,96
...,...,...,...,...,...,...,...
116370,2020-03-17,51800,Suffolk,Western Tidewater,0,0,0
116371,2020-03-17,51810,Virginia Beach,Virginia Beach,4,0,0
116372,2020-03-17,51820,Waynesboro,Central Shenandoah,0,0,0
116373,2020-03-17,51830,Williamsburg,Peninsula,1,0,0


Note that the values represent the total counts since the beginning of data collection in March 2020 until the day in question, and not the total number reported on that day. Next we filter the data to just the most current totals:

In [4]:
cases_today = cases.loc[cases['report_date'] == max(cases['report_date'])]

To calculate the per-capita counts, we merge the data with the population statistics from the U.S. Census that we used in Module 9:

In [5]:
url = "https://demographics.coopercenter.org/sites/demographics/files/media/files/2020-07/Census_2019_RaceEstimates_forVA_0.xls"
pop = pd.read_excel(url, skiprows=4)
pop = pop.loc[~pop['FIPS'].isna()]
pop['FIPS'] = pop['FIPS'] + 51000
pop['FIPS'] = pop['FIPS'].astype('int').astype('str')
pop = pop[['FIPS', 'Total Population']]
pop

,FIPS,Total Population
3,51001,32316.0
4,51003,109330.0
5,51005,14860.0
6,51007,13145.0
7,51009,31605.0
...,...,...
131,51800,92108.0
132,51810,449974.0
133,51820,22630.0
134,51830,14954.0


We merge the population data with the COVID case data and we calculate rates per 1000 people in each city/county:

In [6]:
cases_pop = pd.merge(cases_today, pop, 
                    left_on = ['fips'],
                    right_on = ['FIPS'],
                    how = 'inner')
cases_pop['Cases per 1000 people'] = round(1000*cases_pop['total_cases']/cases_pop['Total Population'],1)
cases_pop['Hospitalizations per 1000 people'] = round(1000*cases_pop['hospitalizations']/cases_pop['Total Population'],1)
cases_pop['Deaths per 1000 people'] = round(1000*cases_pop['deaths']/cases_pop['Total Population'],1)
cases_pop

,report_date,fips,locality,vdh_health_district,total_cases,hospitalizations,deaths,FIPS,Total Population,Cases per 1000 people,Hospitalizations per 1000 people,Deaths per 1000 people
0,2022-08-08,51001,Accomack,Eastern Shore,7866,424,111,51001,32316.0,243.4,13.1,3.4
1,2022-08-08,51003,Albemarle,Blue Ridge,21726,491,176,51003,109330.0,198.7,4.5,1.6
2,2022-08-08,51005,Alleghany,Alleghany,3781,99,101,51005,14860.0,254.4,6.7,6.8
3,2022-08-08,51007,Amelia,Piedmont,3014,171,59,51007,13145.0,229.3,13.0,4.5
4,2022-08-08,51009,Amherst,Central Virginia,8225,356,96,51009,31605.0,260.2,11.3,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...
128,2022-08-08,51800,Suffolk,Western Tidewater,21641,840,313,51800,92108.0,235.0,9.1,3.4
129,2022-08-08,51810,Virginia Beach,Virginia Beach,102988,3464,868,51810,449974.0,228.9,7.7,1.9
130,2022-08-08,51820,Waynesboro,Central Shenandoah,6578,172,69,51820,22630.0,290.7,7.6,3.0
131,2022-08-08,51830,Williamsburg,Peninsula,2012,63,23,51830,14954.0,134.5,4.2,1.5


### A dataframe with the trend of these statistics, defined as the total number in the last two weeks relative to the total number during the two weeks prior to that
We start by reshaping the data to long format to place cases, hospitalizations, and deaths on separate rows:

In [7]:
cases_trend = cases[['report_date', 'fips', 'locality', 'total_cases', 'hospitalizations', 'deaths']]
cases_trend = pd.melt(cases_trend, id_vars = ['report_date', 'fips', 'locality'],
                     value_vars = ['total_cases', 'hospitalizations', 'deaths'])
cases_trend

,report_date,fips,locality,variable,value
0,2022-08-08,51001,Accomack,total_cases,7866
1,2022-08-08,51003,Albemarle,total_cases,21726
2,2022-08-08,51005,Alleghany,total_cases,3781
3,2022-08-08,51007,Amelia,total_cases,3014
4,2022-08-08,51009,Amherst,total_cases,8225
...,...,...,...,...,...
349120,2020-03-17,51800,Suffolk,deaths,0
349121,2020-03-17,51810,Virginia Beach,deaths,0
349122,2020-03-17,51820,Waynesboro,deaths,0
349123,2020-03-17,51830,Williamsburg,deaths,0


To calculate the totals in rolling 14 day windows, we create two additional versions of the date: the date 14 days in the future and the date 28 days in the future. We will use both new dates to merge the dataset to itself. That places the total count on a given day on the same row as the total counts 14 and 28 days ago, and that enables us to create our trend indices:

In [8]:
cases_trend['date14'] = cases_trend['report_date'] + timedelta(14)
cases_trend['date28'] = cases_trend['report_date'] + timedelta(28)
cases_trend

,report_date,fips,locality,variable,value,date14,date28
0,2022-08-08,51001,Accomack,total_cases,7866,2022-08-22,2022-09-05
1,2022-08-08,51003,Albemarle,total_cases,21726,2022-08-22,2022-09-05
2,2022-08-08,51005,Alleghany,total_cases,3781,2022-08-22,2022-09-05
3,2022-08-08,51007,Amelia,total_cases,3014,2022-08-22,2022-09-05
4,2022-08-08,51009,Amherst,total_cases,8225,2022-08-22,2022-09-05
...,...,...,...,...,...,...,...
349120,2020-03-17,51800,Suffolk,deaths,0,2020-03-31,2020-04-14
349121,2020-03-17,51810,Virginia Beach,deaths,0,2020-03-31,2020-04-14
349122,2020-03-17,51820,Waynesboro,deaths,0,2020-03-31,2020-04-14
349123,2020-03-17,51830,Williamsburg,deaths,0,2020-03-31,2020-04-14


Next we merge on the date 14 days in the future:

In [9]:
cases_trend = pd.merge(cases_trend, cases_trend,
                      right_on = ['report_date', 'fips', 'locality', 'variable'],
                      left_on = ['date14', 'fips', 'locality', 'variable'])

cases_trend = cases_trend.drop(['report_date_x','date14_x','date28_x'], axis=1)
cases_trend = cases_trend.rename({'report_date_y':'report_date',
                                 'date14_y':'date14',
                                 'date28_y':'date28',
                                 'value_y':'value',
                                 'value_x':'value14'}, axis=1)
cases_trend

,fips,locality,variable,value14,report_date,value,date14,date28
0,51001,Accomack,total_cases,7714,2022-08-08,7866,2022-08-22,2022-09-05
1,51003,Albemarle,total_cases,21333,2022-08-08,21726,2022-08-22,2022-09-05
2,51005,Alleghany,total_cases,3707,2022-08-08,3781,2022-08-22,2022-09-05
3,51007,Amelia,total_cases,2936,2022-08-08,3014,2022-08-22,2022-09-05
4,51009,Amherst,total_cases,8088,2022-08-08,8225,2022-08-22,2022-09-05
...,...,...,...,...,...,...,...,...
343534,51800,Suffolk,deaths,0,2020-03-31,0,2020-04-14,2020-04-28
343535,51810,Virginia Beach,deaths,0,2020-03-31,2,2020-04-14,2020-04-28
343536,51820,Waynesboro,deaths,0,2020-03-31,0,2020-04-14,2020-04-28
343537,51830,Williamsburg,deaths,0,2020-03-31,1,2020-04-14,2020-04-28


Then we merge on the date 28 days in the future:

In [10]:
cases_trend = pd.merge(cases_trend, cases_trend,
                      right_on = ['report_date', 'fips', 'locality', 'variable'],
                      left_on = ['date28', 'fips', 'locality', 'variable'])

cases_trend = cases_trend.drop(['report_date_x','date14_y','date28_y', 'value14_x', 'date14_x', 'date28_x'], axis=1)
cases_trend = cases_trend.rename({'report_date_y':'report_date',
                                 'value_y':'value',
                                 'value14_y':'value14',
                                 'value_x':'value28'}, axis=1)
cases_trend

,fips,locality,variable,value28,value14,report_date,value
0,51001,Accomack,total_cases,7531,7714,2022-08-08,7866
1,51003,Albemarle,total_cases,20901,21333,2022-08-08,21726
2,51005,Alleghany,total_cases,3618,3707,2022-08-08,3781
3,51007,Amelia,total_cases,2875,2936,2022-08-08,3014
4,51009,Amherst,total_cases,7880,8088,2022-08-08,8225
...,...,...,...,...,...,...,...
332362,51800,Suffolk,deaths,0,1,2020-04-28,8
332363,51810,Virginia Beach,deaths,2,5,2020-04-28,12
332364,51820,Waynesboro,deaths,0,0,2020-04-28,0
332365,51830,Williamsburg,deaths,1,1,2020-04-28,1


Then we create the trend indices. We also create a string version for presentation in a table:

In [11]:
cases_trend['Most recent 14 days'] = cases_trend['value'] - cases_trend['value14']
cases_trend['Previous 14 days'] = cases_trend['value14'] - cases_trend['value28']
cases_trend['Trend'] = 100*(cases_trend['Most recent 14 days'] - cases_trend['Previous 14 days']) / cases_trend['Previous 14 days']
cases_trend['Trend_string'] = round(cases_trend['Trend'], 1).astype('str') + "%"
cases_trend

,fips,locality,variable,value28,value14,report_date,value,Most recent 14 days,Previous 14 days,Trend,Trend_string
0,51001,Accomack,total_cases,7531,7714,2022-08-08,7866,152,183,-16.939891,-16.9%
1,51003,Albemarle,total_cases,20901,21333,2022-08-08,21726,393,432,-9.027778,-9.0%
2,51005,Alleghany,total_cases,3618,3707,2022-08-08,3781,74,89,-16.853933,-16.9%
3,51007,Amelia,total_cases,2875,2936,2022-08-08,3014,78,61,27.868852,27.9%
4,51009,Amherst,total_cases,7880,8088,2022-08-08,8225,137,208,-34.134615,-34.1%
...,...,...,...,...,...,...,...,...,...,...,...
332362,51800,Suffolk,deaths,0,1,2020-04-28,8,7,1,600.000000,600.0%
332363,51810,Virginia Beach,deaths,2,5,2020-04-28,12,7,3,133.333333,133.3%
332364,51820,Waynesboro,deaths,0,0,2020-04-28,0,0,0,NaN,nan%
332365,51830,Williamsburg,deaths,1,1,2020-04-28,1,0,0,NaN,nan%
